In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
from glob import glob

In [1]:
from utils import DQ_TBL, DIM_ORD_STAT, DIM_SELLERS
from utils import PG_USER, PG_PASSWD, PG_HOST, PG_PORT, PG_DB, PG_SCHEMA

Env loaded in utils.


In [ ]:
from EXT_ATH import process_date

In [ ]:
sql_mapping = {
  "base_od_dq_nhm.sql": "dq_main",
  "base_dim_sellers.sql" : "dim_sellers",
  "base_dim_order_status" : "dim_order_status"
}

In [2]:
conn_str = f"postgresql+psycopg://{PG_USER}:{PG_PASSWD}@{PG_HOST}:{PG_PORT}/{PG_DB}"

In [6]:
def get_raw_results(results):
    final_data = []
    columns = [x["VarCharValue"] for x in results["rows"][0]["Data"]]
    rows = [list(map(lambda field: field.get('VarCharValue', ''), row['Data'])) for row in results['rows'][1:]]
    for data in rows:
        final_data.append(data)
    return pd.DataFrame(columns=columns, data=final_data)

In [7]:
def list_dates(start_date, period='days'):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.now().date()
    date_list = []

    while start_date.date() <= end_date:
        date_list.append(start_date.date().strftime(format="%Y-%m-%d"))

        if period == 'days':
            start_date += timedelta(days=1)
        elif period == 'months':
            if start_date.month == 12:
                start_date = start_date.replace(year=start_date.year + 1, month=1)
            else:
                next_month = start_date.month + 1
                try:
                    start_date = start_date.replace(month=next_month)
                except ValueError:
                    # Handle the cases like transitioning from Jan 31 to Feb
                    start_date = start_date.replace(day=1, month=next_month) + timedelta(days=31)
                    start_date = start_date.replace(day=1)
    return date_list
    

In [ ]:
to_drop = [DQ_TBL, DIM_ORD_STAT, DIM_SELLERS]

In [2]:
def exec_sql_stmt(conn_str:str, sql:str):
  engine = create_engine(conn_str)
  with engine.connect() as conn:
    try:
      conn.execute(text(sql))
    except Exception as e:
      raise e
    else:
      print("Execution Successful.")

In [ ]:
for table in to_drop:
  sql_stmt = f"DROP TABLE {PG_SCHEMA}.{table} CASCADE"
  try:
      exec_sql_stmt(conn_str, sql_stmt)
	


In [ ]:
DROP TABLE data_sanity.od_dq_nhm CASCADE;

In [8]:
start_date = '2024-05-01'
dates_between = list_dates(start_date, period="months")

In [ ]:
sql_mapping = {
    "dq_main":"base_od_dq_nhm.sql",
    "dim_sellers":"base_dim_sellers.sql",
    "dim_order_status":"base_dim_order_status"
}

In [ ]:
for date_val in dates_between:
    print(f"Processing {date_val}")
    results = await process_date(tbl_name="nhm_order_fulfillment_subset_v1",date=date_val,raw_query=dq_sql)
    df = get_raw_results(results)
    df.to_parquet(f"D:\\DATA_DUMP\\DATA_QUALITY\\{tbl_name}_{date_val}.parquet", index=False)

In [10]:
tbl_name = "od_dq_nhm"